### Read the Config file from config.py

In [ ]:
!pip install pytorch_lightning
!pip install lightning
!pip install torch
!pip install torchvision
!pip install datasets
!pip install tokenizers
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import OneCycleLR
from config import get_config
from lightning_dataset import BilingualLightning
from lightning_model import TransformerLightning
from lightning_train import train_transformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 596.7/596.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Edit the config file to accomodate the batch_size on available GPU machine

In [ ]:
from config import get_config


In [ ]:
cfg = get_config()

In [ ]:
cfg

{'parameter_sharing': False,
 'd_ff': 2048,
 'batch_size': 32,
 'num_epochs': 30,
 'lr': 0.0001,
 'seq_len': 160,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'fr',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': True,
 'tokenizer_file': 'tokenizer_{0}.json',
 'rundir': 'runs',
 'experiment_name': 'tmodel_dynamic_padding',
 'ds_mode': 'Online',
 'ds_path': None,
 'ds_name': 'opus_books',
 'save_ds_to_disk': True}

In [ ]:
cfg['parameter_sharing'] = True
cfg['d_ff'] = 1024
cfg['batch_size'] = 16
cfg['d_model'] = 512
cfg['preload'] = None
cfg['num_epochs'] = 40
cfg['ds_mode'] = 'Online'
cfg['lang_tgt'] = 'fr'
cfg['save_ds_to_disk'] = False
cfg['experiment_name'] = f"tmodel_dynamic_pad_150_tokens_batch_{cfg['batch_size']}_amp_dff{cfg['d_ff']}"

In [ ]:
# updated config
cfg

{'parameter_sharing': True,
 'd_ff': 1024,
 'batch_size': 16,
 'num_epochs': 40,
 'lr': 0.0001,
 'seq_len': 160,
 'd_model': 512,
 'lang_src': 'en',
 'lang_tgt': 'fr',
 'model_folder': 'weights',
 'model_basename': 'tmodel_',
 'preload': None,
 'tokenizer_file': 'tokenizer_{0}.json',
 'rundir': 'runs',
 'experiment_name': 'tmodel_dynamic_pad_150_tokens_batch_16_amp_dff1024',
 'ds_mode': 'Online',
 'ds_path': None,
 'ds_name': 'opus_books',
 'save_ds_to_disk': False}

### Enable the Mixed Precision Training

In [ ]:
torch.cuda.amp.autocast(enabled=True)

In [ ]:
torch.set_float32_matmul_precision('medium')

### Train the model

In [ ]:
opus_fr = BilingualLightning(cfg)
opus_fr.prepare_data()
opus_fr.setup()
train_data_loader = opus_fr.train_dataloader()

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
tokenizer_src = opus_fr.tokenizer_src
tokenizer_tgt = opus_fr.tokenizer_tgt
loss_criterion = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1)
transformer_model = TransformerLightning(cfg, loss_criterion, tokenizer_src, tokenizer_tgt, num_validation_examples=2, epochs = cfg['num_epochs'])
optimizer = torch.optim.Adam(transformer_model.model.parameters(), lr=cfg['lr'], eps=1e-9)
transformer_model.set_optimizer(optimizer)

scheduler = OneCycleLR(
    optimizer,
    max_lr=1E-3,
    steps_per_epoch=len(train_data_loader),
    epochs=cfg['num_epochs'],
    pct_start=5/cfg['num_epochs'],
    div_factor=10,
    three_phase=True,
    final_div_factor=10,
    anneal_strategy='linear'
)
transformer_model.set_scheduler_dict(scheduler, freq = 'step')

Total Parameters: 61847890


In [ ]:
train_transformer(transformer_model, opus_fr, cfg, epochs=cfg['num_epochs'])

/usr/local/lib/python3.10/dist-packages/lightning_fabric/connector.py:554: UserWarning: 16 is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type             | Params
----------------------------------------------------
0 | loss_criterion | CrossEntropyLoss | 0     
1 | model          | Transformer      | 61.8 M
------------------------------------------

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: Gringoire bent his head to the archdeacon's head and spoke to him in a very low voice, casting an uneasy glance the while from one end to the other of the street, though no one was passing.
    TARGET: Gringoire se pencha à l’oreille de l’archidiacre et lui parla très bas, en jetant un regard inquiet d’un bout à l’autre de la rue où il ne passait pourtant personne.
 PREDICTED: Gringoire se leva sa tête de la tête et lui dit d ’ un rire de voix , se mit à la voix , tandis que la voix de la porte , sans doute , sans doute , sans doute .
*****************************************

*****************************************
    SOURCE: The collection of reports of the Grand Army and the _Memorial de Sainte-Helene_ completed his Koran.
    TARGET: Le recueil des bulletins de la grande armée et le Mémorial de Sainte-Hélène complétaient son Coran.
 PREDICTED: Les de l ’ de l ’ et le de sa .
*****************************************



/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `CharErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `CharErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `WordErrorRate` from `torchmetrics` was deprecated and will be removed in 2.0. Import `WordErrorRate` from `torchmetrics.text` instead.
  _future_warning(
/usr/local/lib/python3.10/dist-packages/torchmetrics/utilities/prints.py:62: FutureWarning: Importing `BLEUScore` from `torchmetrics` was deprecated and will be removed in 2.0. Import `BLEUScore` from `torchmetrics.text` instead.
  _future_warning(


...

*****************************************
    SOURCE: He bit small pieces from it with his teeth, and she chewed them, and endeavoured to swallow them.
    TARGET: Il en coupa de petits morceaux avec les dents, et elle les broyait, s'acharnait a les avaler.
 PREDICTED: Il s ' en alla , les petits morceaux , elle les , tenta de les .
*****************************************

*****************************************
    SOURCE: A big yard, overgrown by weeds where children seemed to have come to play during the long dragging evenings at the end of winter, was hollowed out by the rain.
    TARGET: Une grande cour herbeuse, où des enfants avaient dû venir pendant les longues et lentes soirées de la fin de l’hiver, était ravinée par l’orage.
 PREDICTED: Une grande cour d ’ arbres , par les herbes où les enfants paraissaient jouer aux longues soirées du fond de l ’ hiver , s ’ était par la pluie .
*****************************************

Epoch 19: train_loss=1.7670


Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: Having shut the door, M. de Renal seated himself with gravity.
    TARGET: La porte fermée, M. de Rênal s’assit avec gravité.
 PREDICTED: Après avoir fermé la porte , M . de Rênal s ’ assit avec gravité .
*****************************************

*****************************************
    SOURCE: "Have you got it?"
    TARGET: --Vous les avez?
 PREDICTED: -- L ' avez - vous tiré ?
*****************************************

Epoch 20: train_loss=1.9241


Validation: 0it [00:00, ?it/s]

*****************************************
    SOURCE: La Chantefleurie had been a poor creature during the five years since her fall.
    TARGET: Depuis cinq ans qu’elle avait failli, c’était une pauvre créature que la Chantefleurie.
 PREDICTED: La Chantefleurie avait été pauvre créature pendant cinq ans qu ’ elle tomba .
*****************************************

*****************************************
    SOURCE: They regarded her as a distraction that drew them from their bad dreams.
    TARGET: Ils la considéraient comme une distraction qui les tirait de leurs mauvais rêves.
 PREDICTED: Ils la regardaient comme une distraction qu ’ ils la de leurs mauvais rêves .
*****************************************

Epoch 21: train_loss=1.7949


### Sample Predictions using the trained model

In [ ]:
def encode_input_sentence(input, tokenizer_src, max_seq_len) -> torch.tensor:
    encoder_input_tokens = tokenizer_src.encode(input).ids
    sos_token = torch.tensor([tokenizer_src.token_to_id("[SOS]")], dtype=torch.int64)
    eos_token = torch.tensor([tokenizer_src.token_to_id("[EOS]")], dtype=torch.int64)
    pad_token = torch.tensor([tokenizer_src.token_to_id("[PAD]")], dtype=torch.int64)
    # Add sos, eos and padding to each sentence
    enc_num_padding_tokens = max_seq_len - len(encoder_input_tokens) - 2 # we will add <s> and </s>

    encoder_input = torch.cat(
        [
            sos_token,
            torch.tensor(encoder_input_tokens, dtype=torch.int64),
            eos_token,
            torch.tensor([pad_token]*enc_num_padding_tokens, dtype=torch.int64)
        ],
        dim = 0)
    encoder_mask = (encoder_input != pad_token).unsqueeze(0).unsqueeze(0).int()
    return encoder_input, encoder_mask

def translate_en_to_fr(input, opus_dm, cfg):
    encoder_input, encoder_mask = encode_input_sentence(input, opus_dm.tokenizer_src, cfg['seq_len'])
    output = transformer_model.greedy_decode(encoder_input, encoder_mask)
    model_out_text = opus_dm.tokenizer_tgt.decode(output.detach().cpu().numpy())
    return model_out_text

In [ ]:
input = "The firm resolution to commit another murder somewhat calmed them."
output = translate_en_to_fr(input, opus_fr, cfg)

print(output)

### Logs in TensorBoard

In [ ]:

%load_ext tensorboard
%tensorboard --logdir='runs'